In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from collections import defaultdict



In [4]:
df_trips = pd.read_csv("data/trips.txt")
df_routes = pd.read_csv("data/routes.txt")
df_stops = pd.read_csv("data/stops.txt")
df_stop_times = pd.read_csv("data/stop_times.txt")
df_pathways = pd.read_csv("data/pathways.txt")

/var/folders/16/qc8wq8c93v3f89ksks8p_szm0000gn/T/ipykernel_41291/98565878.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trips = pd.read_csv("data/trips.txt")
/var/folders/16/qc8wq8c93v3f89ksks8p_szm0000gn/T/ipykernel_41291/98565878.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stop_times = pd.read_csv("data/stop_times.txt")


# Egy megállóból elérhető szomszédok keresése

## Járatok

In [5]:
df_trips.head(5)

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,8140,C0418310,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_5,Y701,1.0,1.0
1,8140,C0418311,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_6,Y702,1.0,1.0
2,8140,C0418312,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_7,Y701,1.0,1.0
3,8140,C0418313,C04183AHPMAA-011,Csepel-Királyerdő,1,C04183_8140_1_8,Y702,1.0,1.0
4,8140,C0418314,C04183AHPMAA-011,"Soroksár, Molnár-sziget",0,C04183_8140_1_9,Y701,1.0,1.0


## Melyik járat, melyik megállóban mikor áll meg?

In [6]:
df_stop_times.head(5)


,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,C0418310,F04181,08:03:00,08:03:00,0,NaN,NaN,NaN,0.0
1,C0418310,F04526,08:06:00,08:06:00,1,NaN,NaN,NaN,160.0
2,C0418311,F04526,08:30:00,08:30:00,0,NaN,NaN,NaN,0.0
3,C0418311,F04181,08:33:00,08:33:00,1,NaN,NaN,NaN,160.0
4,C0418312,F04181,08:33:00,08:33:00,0,NaN,NaN,NaN,0.0


## Most a string arrival_time-ből csináljunk számokat
Mert igy jóval gyorsabb lesz a lekérdezés

In [7]:
def str_time_to_min(astr):
    [h,m,s]=astr.split(':')
    return int(h)*60+int(m)


In [8]:
df_stop_times["arrival_time_min"]=df_stop_times["arrival_time"].apply(str_time_to_min)
df_stop_times["departure_time_min"]=df_stop_times["departure_time"].apply(str_time_to_min)

df_stop_times[["trip_id","stop_id","arrival_time","stop_sequence","arrival_time_min","departure_time_min"]].head(5)

,trip_id,stop_id,arrival_time,stop_sequence,arrival_time_min,departure_time_min
0,C0418310,F04181,08:03:00,0,483,483
1,C0418310,F04526,08:06:00,1,486,486
2,C0418311,F04526,08:30:00,0,510,510
3,C0418311,F04181,08:33:00,1,513,513
4,C0418312,F04181,08:33:00,0,513,513


Mennyi ideig kell sétálni?

In [9]:
def getminutestowalk(dst,walkspeed=3.5):
    # km/h to m / minutes
    walkspeed = walkspeed * 1000/60
    return dst / walkspeed

In [10]:
getminutestowalk(1000)

17.142857142857142

## Adott megállóban melyik járatok állnak meg X időpont után

"008152", # Mester utca / könyveskalman
"008163" # Újpest Városkapu
"F00191", # Margit híd, budai hídfő H
"F01083" # Király utca / Erzsébet körút
"F01755", # Örs vezér tere M+H'
"F01083" #  Oktogon M
"F02268", # KFKI
"009019" # Gödöllő

In [11]:
stop_id = "F00191"
#stop_id = "F00189"
arrival_time_min=str_time_to_min("9:01:00")
arrival_time_min

541

In [12]:
a=df_stops.query(f'stop_id == "{stop_id}"')["stop_name"]
stop_name=a.values[0]
stop_name

'Margit híd, budai hídfő H'

In [13]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time_min > {arrival_time_min}'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time_min"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name
df_megallo_jaratok[["stop_id","stop_name","trip_id", "route_id","direction_id","arrival_time", "stop_headsign","stop_sequence"]]


,stop_id,stop_name,trip_id,route_id,direction_id,arrival_time,stop_headsign,stop_sequence
222,F00191,"Margit híd, budai hídfő H",D0576812124,1910,0,09:02:00,NaN,2
649,F00191,"Margit híd, budai hídfő H",D0752910466,1910,0,09:02:00,Szemlőhegy,2
111,F00191,"Margit híd, budai hídfő H",D057653577,2910,0,09:05:00,"Zugliget, Libegő",2
491,F00191,"Margit híd, budai hídfő H",D0702614936,2910,0,09:06:00,"Zugliget, Libegő",2
112,F00191,"Margit híd, budai hídfő H",D057653587,910,0,09:15:00,Rózsadomb ► Széll Kálmán tér M,2
651,F00191,"Margit híd, budai hídfő H",D0752910476,0910,0,09:16:00,Rózsadomb ► Széll Kálmán tér M,2
930,F00191,"Margit híd, budai hídfő H",D08676569,VP06,0,23:33:00,NaN,11


In [58]:
df_megallo_jaratok2 = pd.merge( df_megallo_jaratok, df_routes[["route_id", "route_short_name"]], left_on="route_id", right_on="route_id")
df_megallo_jaratok2[["stop_id","stop_name","trip_id", "route_id","direction_id","route_short_name", "arrival_time", "arrival_time_min", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,trip_id,route_id,direction_id,route_short_name,arrival_time,arrival_time_min,stop_headsign,stop_sequence
0,048506,Szent István Bazilika,C96610211,4720,0,72,10:01:00,601,Zugló vasútállomás,9
1,048506,Szent István Bazilika,D07644658,0090,1,9,10:05:00,605,"Óbuda, Bogdáni út",17
2,048506,Szent István Bazilika,D079871,F4720,0,72❆,10:32:00,632,Zugló vasútállomás,9
3,048506,Szent István Bazilika,D0781122,9500,1,950,24:09:00,1449,"Rákospalota, Székely Elek út",12
4,048506,Szent István Bazilika,D077561222,9140,1,914,24:19:00,1459,"Káposztásmegyer, Mogyoródi-patak",30


## Kezdjük el keresi a megálló szomszédait

### stop_times mintha erre lenne kitalálva, mert valójában egy trip későbbi megállóit keresem!

In [59]:
trip_id="D063983577"
#trip_id = "D015152828"
#trip_id = "D057685654"
arrival_time_min = 541

In [60]:
df_stop_times.query(f'trip_id == "{trip_id}" and arrival_time_min > {arrival_time_min}')

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min


# Járattal elérhető szomszédok meghatározása

Először valamilyen ciklusra gondoltam, de az inner join sokkal jobb

Előről kezdem és megpróbálom a részfolyamatokat összeilleszteni

In [61]:
stop_id = "F01034"
stop_id = "F00191"
stop_id = "F00191"
#stop_id = "F00189"
stop_id = "19795278"
#stop_id = "19795279"
arrival_time_min=str_time_to_min("7:01:00")
arrival_time_min

421

In [62]:
a=df_stops.query(f'stop_id == "{stop_id}"')["stop_name"]
stop_name=a.values[0]
stop_name

'Örs vezér tere'

Melyik járatok állnak meg itt X időpont után:

In [63]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time_min > {arrival_time_min}'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time_min"])
df_megallo_jaratok


,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
10,H101722_38,19795278,07:08:00,07:08:00,20,NaN,1.0,NaN,24728.0,428,428,H8,60950,Örs vezér tere,1,NaN,38,NaN,1.0
11,H101725_38,19795278,07:08:00,07:08:00,20,NaN,1.0,NaN,24728.0,428,428,H8,60995,Örs vezér tere,1,NaN,38,NaN,1.0
33,H104224_37,19795278,07:15:00,07:15:00,1,NaN,NaN,1.0,0.0,435,435,H8,60952,Gödöllő,0,NaN,37,NaN,1.0
17,H102728_38,19795278,07:24:00,07:24:00,20,NaN,1.0,NaN,24728.0,444,444,H8,60994,Örs vezér tere,1,NaN,38,NaN,1.0
16,H102725_38,19795278,07:24:00,07:24:00,20,NaN,1.0,NaN,24728.0,444,444,H8,60950,Örs vezér tere,1,NaN,38,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,H108126_38,19795278,24:14:00,24:14:00,20,NaN,1.0,NaN,24728.0,1454,1454,H8,60959,Örs vezér tere,1,NaN,38,NaN,1.0
66,H108129_38,19795278,24:14:00,24:14:00,20,NaN,1.0,NaN,24728.0,1454,1454,H8,60990,Örs vezér tere,1,NaN,38,NaN,1.0
74,H109127_38,19795278,24:14:00,24:14:00,20,NaN,1.0,NaN,24728.0,1454,1454,H8,60993,Örs vezér tere,1,NaN,38,NaN,1.0
83,H110113_38,19795278,24:14:00,24:14:00,20,NaN,1.0,NaN,24728.0,1454,1454,H8,60950,Örs vezér tere,1,NaN,38,NaN,1.0


Itt majd kitörlöm az ismétlődéseket

In [64]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time_min > {arrival_time_min}'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time_min"]).drop_duplicates(subset=["route_id","direction_id"], keep='first')


In [65]:
df_megallo_jaratok

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
10,H101722_38,19795278,07:08:00,07:08:00,20,NaN,1.0,NaN,24728.0,428,428,H8,60950,Örs vezér tere,1,NaN,38,NaN,1.0
33,H104224_37,19795278,07:15:00,07:15:00,1,NaN,NaN,1.0,0.0,435,435,H8,60952,Gödöllő,0,NaN,37,NaN,1.0
146,H15044_43,19795278,12:18:00,12:18:00,1,NaN,NaN,1.0,0.0,738,738,H9,60936,Csömör,0,NaN,43,NaN,1.0
147,H15059_44,19795278,12:39:00,12:39:00,12,NaN,1.0,NaN,10599.0,759,759,H9,60936,Örs vezér tere,1,NaN,44,NaN,1.0


In [67]:
#df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name
df_megallo_jaratok[["stop_id","stop_name","trip_id", "route_id","arrival_time", "arrival_time_min", "trip_headsign","stop_sequence"]]

,stop_id,stop_name,trip_id,route_id,arrival_time,arrival_time_min,trip_headsign,stop_sequence
10,19795278,Örs vezér tere,H101722_38,H8,07:08:00,428,Örs vezér tere,20
33,19795278,Örs vezér tere,H104224_37,H8,07:15:00,435,Gödöllő,1
146,19795278,Örs vezér tere,H15044_43,H9,12:18:00,738,Csömör,1
147,19795278,Örs vezér tere,H15059_44,H9,12:39:00,759,Örs vezér tere,12


Kozmetika, hogy legyen járatszám amit értek is

In [69]:
df_megallo_jaratok2 = pd.merge( df_megallo_jaratok, df_routes[["route_id","route_short_name"]], left_on="route_id", right_on="route_id")
df_megallo_jaratok2[["stop_id","stop_name","trip_id", "route_id","route_short_name", "arrival_time", "arrival_time_min", "trip_headsign","stop_sequence"]]

,stop_id,stop_name,trip_id,route_id,route_short_name,arrival_time,arrival_time_min,trip_headsign,stop_sequence
0,19795278,Örs vezér tere,H101722_38,H8,H8,07:08:00,428,Örs vezér tere,20
1,19795278,Örs vezér tere,H104224_37,H8,H8,07:15:00,435,Gödöllő,1
2,19795278,Örs vezér tere,H15044_43,H9,H9,12:18:00,738,Csömör,1
3,19795278,Örs vezér tere,H15059_44,H9,H9,12:39:00,759,Örs vezér tere,12


### Szomszédokat egy left join-nal fogok keresni

In [70]:
df_szomszedok = pd.merge( df_megallo_jaratok2, df_stop_times[["trip_id","stop_id","arrival_time_min","arrival_time","departure_time","departure_time_min"]], how="left", left_on="trip_id", right_on="trip_id")
df_szomszedok.columns

Index(['trip_id', 'stop_id_x', 'arrival_time_x', 'departure_time_x',
       'stop_sequence', 'stop_headsign', 'pickup_type', 'drop_off_type',
       'shape_dist_traveled', 'arrival_time_min_x', 'departure_time_min_x',
       'route_id', 'service_id', 'trip_headsign', 'direction_id', 'block_id',
       'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'stop_name',
       'route_short_name', 'stop_id_y', 'arrival_time_min_y', 'arrival_time_y',
       'departure_time_y', 'departure_time_min_y'],
      dtype='object')

Túl korai járatokat törlöm

In [71]:
df_szomszedok.rename(columns={"stop_name": "stop_name_x", "arrival_time_y":"arrival_time", "arrival_time_min_y":"arrival_time_min", "stop_id_y": "stop_id", }, inplace=True)

In [26]:
#indices_to_drop = df_szomszedok [ df_szomszedok["arrival_time_min"] <= arrival_time_min ].index
#df_szomszedok.drop( indices_to_drop, inplace= True)

Saját magamat sem akarom látni mint szomszédot

In [27]:
#indices_to_drop = df_szomszedok [ df_szomszedok["stop_id_x"] == df_szomszedok["stop_id"] ].index
#df_szomszedok.drop( indices_to_drop, inplace= True)

In [77]:
df_szomszedok[["trip_id","stop_id_x","arrival_time_x","stop_name_x","stop_id","arrival_time_min"]]

,trip_id,stop_id_x,arrival_time_x,stop_name_x,stop_id,arrival_time_min
0,H101722_38,19795278,07:08:00,Örs vezér tere,19870325,382
1,H101722_38,19795278,07:08:00,Örs vezér tere,19869323,383
2,H101722_38,19795278,07:08:00,Örs vezér tere,19868322,386
3,H101722_38,19795278,07:08:00,Örs vezér tere,19867320,388
4,H101722_38,19795278,07:08:00,Örs vezér tere,19865318,394
...,...,...,...,...,...,...
59,H15059_44,19795278,12:39:00,Örs vezér tere,19824288,751
60,H15059_44,19795278,12:39:00,Örs vezér tere,19823286,753
61,H15059_44,19795278,12:39:00,Örs vezér tere,19810284,754
62,H15059_44,19795278,12:39:00,Örs vezér tere,19798282,756


In [73]:
df_szomszedok2 = pd.merge(df_szomszedok, df_stops[["stop_id","stop_name","stop_lat","stop_lon"]], how = "left", left_on="stop_id", right_on="stop_id")
df_szomszedok2.columns

Index(['trip_id', 'stop_id_x', 'arrival_time_x', 'departure_time_x',
       'stop_sequence', 'stop_headsign', 'pickup_type', 'drop_off_type',
       'shape_dist_traveled', 'arrival_time_min_x', 'departure_time_min_x',
       'route_id', 'service_id', 'trip_headsign', 'direction_id', 'block_id',
       'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'stop_name_x',
       'route_short_name', 'stop_id', 'arrival_time_min', 'arrival_time',
       'departure_time_y', 'departure_time_min_y', 'stop_name', 'stop_lat',
       'stop_lon'],
      dtype='object')

In [74]:
df_szomszedok2[["trip_id","route_short_name", "stop_id_x","stop_name_x","departure_time_x","arrival_time_x", "arrival_time_min_x", "stop_id", "stop_name", "arrival_time","arrival_time_min", "stop_lon","stop_lat"]]


,trip_id,route_short_name,stop_id_x,stop_name_x,departure_time_x,arrival_time_x,arrival_time_min_x,stop_id,stop_name,arrival_time,arrival_time_min,stop_lon,stop_lat
0,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,428,19870325,Gödöllő,06:22:00,382,19.357787,47.593341
1,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,428,19869323,"Gödöllő, Palotakert",06:23:00,383,19.356661,47.598525
2,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,428,19868322,"Gödöllő, Szabadság tér",06:26:00,386,19.348252,47.597827
3,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,428,19867320,"Gödöllő, Erzsébet park",06:28:00,388,19.340771,47.595072
4,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,428,19865318,Szentjakab,06:34:00,394,19.290352,47.597338
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,H15059_44,H9,19795278,Örs vezér tere,12:39:00,12:39:00,759,19824288,"Mátyásföld, repülőtér",12:31:00,751,19.194054,47.511857
60,H15059_44,H9,19795278,Örs vezér tere,12:39:00,12:39:00,759,19823286,Sashalom,12:33:00,753,19.178647,47.508952
61,H15059_44,H9,19795278,Örs vezér tere,12:39:00,12:39:00,759,19810284,Nagyicce,12:34:00,754,19.167690,47.507017
62,H15059_44,H9,19795278,Örs vezér tere,12:39:00,12:39:00,759,19798282,Rákosfalva,12:36:00,756,19.151378,47.505067


In [75]:
indices_to_drop = df_szomszedok2 [ df_szomszedok2["arrival_time_min_x"] > df_szomszedok2["arrival_time_min"] ].index
df_szomszedok2.drop( indices_to_drop, inplace= True)

In [76]:
df_szomszedok2[["trip_id","route_short_name", "stop_id_x","stop_name_x","departure_time_x", "arrival_time_x","stop_id", "stop_name", "arrival_time","stop_lon","stop_lat"]]

,trip_id,route_short_name,stop_id_x,stop_name_x,departure_time_x,arrival_time_x,stop_id,stop_name,arrival_time,stop_lon,stop_lat
19,H101722_38,H8,19795278,Örs vezér tere,07:08:00,07:08:00,19795278,Örs vezér tere,07:08:00,19.137192,47.503585
20,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19795278,Örs vezér tere,07:15:00,19.137192,47.503585
21,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19798281,Rákosfalva,07:17:00,19.151378,47.505067
22,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19810283,Nagyicce,07:19:00,19.167690,47.507017
23,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19823285,Sashalom,07:21:00,19.178647,47.508952
24,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19824287,"Mátyásföld, repülőtér",07:23:00,19.194054,47.511857
25,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19825289,"Mátyásföld, Imre utca",07:24:00,19.199582,47.513546
26,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19826291,Mátyásföld alsó,07:26:00,19.207456,47.515986
27,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19828293,Cinkota,07:28:00,19.217338,47.519435
28,H104224_37,H8,19795278,Örs vezér tere,07:15:00,07:15:00,19841302,Ilonatelep,07:31:00,19.229403,47.526627


In [80]:
df_szomszedok2.drop_duplicates(["stop_id"],inplace=True)
df_szomszedok2[["trip_id","route_short_name", "trip_headsign", "stop_id_x","stop_name_x","departure_time_x", "arrival_time_x","stop_id", "stop_name", "arrival_time","stop_lon","stop_lat"]]

,trip_id,route_short_name,trip_headsign,stop_id_x,stop_name_x,departure_time_x,arrival_time_x,stop_id,stop_name,arrival_time,stop_lon,stop_lat
19,H101722_38,H8,Örs vezér tere,19795278,Örs vezér tere,07:08:00,07:08:00,19795278,Örs vezér tere,07:08:00,19.137192,47.503585
21,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19798281,Rákosfalva,07:17:00,19.151378,47.505067
22,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19810283,Nagyicce,07:19:00,19.167690,47.507017
23,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19823285,Sashalom,07:21:00,19.178647,47.508952
24,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19824287,"Mátyásföld, repülőtér",07:23:00,19.194054,47.511857
25,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19825289,"Mátyásföld, Imre utca",07:24:00,19.199582,47.513546
26,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19826291,Mátyásföld alsó,07:26:00,19.207456,47.515986
27,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19828293,Cinkota,07:28:00,19.217338,47.519435
28,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19841302,Ilonatelep,07:31:00,19.229403,47.526627
29,H104224_37,H8,Gödöllő,19795278,Örs vezér tere,07:15:00,07:15:00,19843304,"Kistarcsa, kórház",07:34:00,19.249034,47.539065


In [33]:
df_szomszedok2[["stop_id", "trip_id", "route_short_name", "stop_name", "departure_time_x","arrival_time","stop_lon","stop_lat"]].values

array([['19795278', 'H101722_38', 'H8', 'Örs vezér tere', '07:08:00',
        '07:08:00', 19.137192, 47.503585],
       ['19795278', 'H15044_43', 'H9', 'Örs vezér tere', '12:18:00',
        '12:18:00', 19.137192, 47.503585],
       ['19798281', 'H15044_43', 'H9', 'Rákosfalva', '12:18:00',
        '12:20:00', 19.151378, 47.505067],
       ['19810283', 'H15044_43', 'H9', 'Nagyicce', '12:18:00',
        '12:22:00', 19.16769, 47.507017],
       ['19823285', 'H15044_43', 'H9', 'Sashalom', '12:18:00',
        '12:24:00', 19.178647, 47.508952],
       ['19824287', 'H15044_43', 'H9', 'Mátyásföld, repülőtér',
        '12:18:00', '12:26:00', 19.194054, 47.511857],
       ['19825289', 'H15044_43', 'H9', 'Mátyásföld, Imre utca',
        '12:18:00', '12:27:00', 19.199582, 47.513546],
       ['19826291', 'H15044_43', 'H9', 'Mátyásföld alsó', '12:18:00',
        '12:29:00', 19.207456, 47.515986],
       ['19828293', 'H15044_43', 'H9', 'Cinkota', '12:18:00', '12:31:00',
        19.217338, 47.519435],


In [34]:
#retdict = defaultdict(list)
retdict=df_szomszedok2[["stop_id", "trip_id", "route_short_name", "stop_name", "departure_time_min_x", "arrival_time_min","stop_lon","stop_lat"]].to_dict('records')
retdict

[{'stop_id': '19795278',
  'trip_id': 'H101722_38',
  'route_short_name': 'H8',
  'stop_name': 'Örs vezér tere',
  'departure_time_min_x': 428,
  'arrival_time_min': 428,
  'stop_lon': 19.137192,
  'stop_lat': 47.503585},
 {'stop_id': '19795278',
  'trip_id': 'H15044_43',
  'route_short_name': 'H9',
  'stop_name': 'Örs vezér tere',
  'departure_time_min_x': 738,
  'arrival_time_min': 738,
  'stop_lon': 19.137192,
  'stop_lat': 47.503585},
 {'stop_id': '19798281',
  'trip_id': 'H15044_43',
  'route_short_name': 'H9',
  'stop_name': 'Rákosfalva',
  'departure_time_min_x': 738,
  'arrival_time_min': 740,
  'stop_lon': 19.151378,
  'stop_lat': 47.505067},
 {'stop_id': '19810283',
  'trip_id': 'H15044_43',
  'route_short_name': 'H9',
  'stop_name': 'Nagyicce',
  'departure_time_min_x': 738,
  'arrival_time_min': 742,
  'stop_lon': 19.16769,
  'stop_lat': 47.507017},
 {'stop_id': '19823285',
  'trip_id': 'H15044_43',
  'route_short_name': 'H9',
  'stop_name': 'Sashalom',
  'departure_time_mi

## Keressük meg egy útvonalon a megállók neveit

Megoldási terv: Keresek egy routet-t és az első trip-et benne.

Ahhoz a triphez lekérdezzük a megállókat.
Utána majd mellétesszük a neveket is



In [35]:
df_trips.groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})

,route_id,trip_id,trip_headsign
route_id,,,
50,50,D01676116,Pasaréti tér
70,70,D06143101,"Újpalota, Nyírpalota út"
75,75,D0172110,Blaha Lujza tér M
90,90,D05740102,"Óbuda, Bogdáni út"
110,110,D0617810,Batthyány tér M H
...,...,...,...
H6,H6,H20002_11,Ráckeve
H7,H7,H30005_5,Csepel
H8,H8,H100027_37,Gödöllő


### Melyik megállókban áll meg a 4-6-os ?

#### Mikor indul reggel a 4-6-os

A 3060-as járat első indulását keresem.
A groupby és agg funckiók miatt fontos

In [36]:
df_trips[ df_trips["route_id"].isin(["3060"]) ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"]

route_id
3060    D015151403
Name: trip_id, dtype: object

Megvan a trip_id

In [37]:
df_stop_times[ df_stop_times["trip_id"] == "D015151066"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min
1155147,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0,347,347
1155148,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0,349,349
1155149,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0,350,350
1155150,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0,351,351
1155151,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0,353,353
1155152,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0,355,355
1155153,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0,357,357
1155154,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0,358,358
1155155,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0,360,360
1155156,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0,361,361


### Most tegyük mellé a neveket is!

A hátsó stop_name oszlopban látszik

In [38]:
pd.merge( df_stop_times[ df_stop_times["trip_id"] == "D015151066"], df_stops, on="stop_id")

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding
0,D015151066,F01998,05:47:00,05:47:00,0,Széll Kálmán tér M,NaN,NaN,0.0,347,347,Újbuda-központ M,47.474042,19.046862,F01998,NaN,NaN,NaN,1.0
1,D015151066,F01992,05:49:00,05:49:00,1,Széll Kálmán tér M,NaN,NaN,544.0,349,349,Budafoki út / Szerémi sor,47.474114,19.053747,F01992,NaN,NaN,NaN,1.0
2,D015151066,F02225,05:50:00,05:50:00,2,Széll Kálmán tér M,NaN,NaN,1087.0,350,350,"Petőfi híd, budai hídfő",47.476956,19.059572,F02225,NaN,NaN,NaN,1.0
3,D015151066,F01374,05:51:00,05:51:00,3,Széll Kálmán tér M,NaN,NaN,1750.0,351,351,Boráros tér H,47.480400,19.066697,F01374,NaN,NaN,NaN,2.0
4,D015151066,F01380,05:53:00,05:53:00,4,Széll Kálmán tér M,NaN,NaN,2073.0,353,353,Mester utca / Ferenc körút,47.482775,19.068848,F01380,NaN,NaN,NaN,1.0
5,D015151066,F01191,05:55:00,05:55:00,5,Széll Kálmán tér M,NaN,NaN,2395.0,355,355,Corvin-negyed M,47.485500,19.069924,F01191,NaN,NaN,NaN,1.0
6,D015151066,F01199,05:57:00,05:57:00,6,Széll Kálmán tér M,NaN,NaN,2948.0,357,357,Harminckettesek tere,47.490357,19.070841,F01199,NaN,NaN,NaN,1.0
7,D015151066,F01200,05:58:00,05:58:00,7,Széll Kálmán tér M,NaN,NaN,3233.0,358,358,Rákóczi tér M,47.492866,19.071187,F01200,NaN,NaN,NaN,1.0
8,D015151066,F01168,06:00:00,06:00:00,8,Széll Kálmán tér M,NaN,NaN,3677.0,360,360,Blaha Lujza tér M,47.496770,19.070725,F01168,NaN,NaN,NaN,1.0
9,D015151066,F01111,06:01:00,06:01:00,9,Széll Kálmán tér M,NaN,NaN,4109.0,361,361,Wesselényi utca / Erzsébet körút,47.500350,19.068934,F01111,NaN,NaN,NaN,1.0


# Kódrészlet ami egy vonal megállóit keresi meg
Most egy kis kódrészlet keresi nekem az adott vonal megállóit.
Ezt lehet majd beletenni egy függvénybe

In [39]:
#route_id = "3060"
#route_id="3040"
route_id="0090"
elso_trip = df_trips[ df_trips["route_id"] == route_id ].groupby(["route_id"]).agg({"route_id":"first","trip_id":"first", "trip_headsign":"first"})["trip_id"].iloc[0]
a=pd.merge( df_stop_times[ df_stop_times["trip_id"] == elso_trip], df_stops, on="stop_id")
a["route_id"]=route_id
a[["route_id","trip_id","stop_id","stop_name","arrival_time","departure_time","stop_lat","stop_lon"]]

,route_id,trip_id,stop_id,stop_name,arrival_time,departure_time,stop_lat,stop_lon
0,0090,D0764412,F01636,Kőbánya alsó vasútállomás,05:12:00,05:12:00,47.483652,19.127839
1,0090,D0764412,F01653,Szent László tér,05:13:00,05:13:00,47.485925,19.131093
2,0090,D0764412,F01641,Liget tér,05:15:00,05:15:00,47.482795,19.130358
3,0090,D0764412,F01571,Kőbánya alsó vasútállomás (Mázsa tér),05:17:00,05:17:00,47.482879,19.125715
4,0090,D0764412,F01563,Egészségház,05:18:00,05:18:00,47.484126,19.116668
5,0090,D0764412,F01733,Eiffel Műhelyház,05:19:00,05:19:00,47.485136,19.112914
6,0090,D0764412,F01267,Kőbányai út / Könyves Kálmán körút,05:21:00,05:21:00,47.486740,19.106971
7,0090,D0764412,008036,Kőbányai út 31.,05:21:00,05:21:00,47.487461,19.104252
8,0090,D0764412,F01187,Orczy tér,05:23:00,05:24:00,47.489470,19.090495
9,0090,D0764412,F01223,Kálvária tér,05:25:00,05:25:00,47.488890,19.085146


## Melyik járatok állnak meg ebben a megállóban ?

In [40]:
df_stop_times[ df_stop_times["stop_id"] == "048506"]

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,arrival_time_min,departure_time_min
566546,C96610106,048506,07:27:00,07:27:00,9,Zugló vasútállomás,NaN,NaN,3713.0,447,447
566598,C96610111,048506,07:37:00,07:37:00,9,Zugló vasútállomás,NaN,NaN,3713.0,457,457
566648,C96610116,048506,07:44:00,07:44:00,9,Zugló vasútállomás,NaN,NaN,3713.0,464,464
566700,C96610121,048506,07:52:00,07:52:00,9,Zugló vasútállomás,NaN,NaN,3713.0,472,472
566725,C966101221,048506,16:35:00,16:35:00,9,Zugló vasútállomás,NaN,NaN,3713.0,995,995
...,...,...,...,...,...,...,...,...,...,...,...
6040371,D0875684,048506,08:34:00,08:34:00,17,NaN,NaN,NaN,8094.0,514,514
6040436,D0875687,048506,08:42:00,08:42:00,17,NaN,NaN,NaN,8094.0,522,522
6040501,D087569,048506,05:25:00,05:25:00,17,NaN,NaN,NaN,8094.0,325,325
6040533,D0875690,048506,08:51:00,08:51:00,17,NaN,NaN,NaN,8094.0,531,531


Látszik, hogy jó sok indulás van ebből a megállóból

## Adatvizsgálat: hogyan lesznek meg a fő megállók?


In [41]:
stop_id = "048506" # Szent István Bazilika
#stop_id = "F01029" # Nyugati
parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

In [42]:
if parent_stop_id is not np.nan:
        stop_id = parent_stop_id
stop_name = df_stops[ df_stops["stop_id"] == stop_id]["stop_name"].iloc[0]
stop_name

'Szent István Bazilika'

Itt látszik, hogy ez igy nem lesz OK. Az adatok nem tartalmaznak információt arra, hogy melyik az al, vagy melyik a fő megálló

## Melyik útvonalak állnak meg ebben a megállóban?


In [43]:
df_megallo_jaratok = pd.merge( df_stop_times[ df_stop_times["stop_id"] == stop_id], df_trips, left_on="trip_id", right_on="trip_id").drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","stop_headsign","stop_sequence"]]


,stop_id,stop_name,route_id,stop_headsign,stop_sequence
0,048506,Szent István Bazilika,4720,Zugló vasútállomás,9
544,048506,Szent István Bazilika,9310,Nyugati pályaudvar M ► Zöldmál,2
567,048506,Szent István Bazilika,9310,Nyugati pályaudvar M ► Zöldmál,2
572,048506,Szent István Bazilika,9500,"Rákospalota, Székely Elek út",42
573,048506,Szent István Bazilika,9501,"Rákospalota, Kossuth utca",42
594,048506,Szent István Bazilika,90,NaN,17
925,048506,Szent István Bazilika,9500,"Rákospalota, Székely Elek út",42
926,048506,Szent István Bazilika,9501,NaN,42
939,048506,Szent István Bazilika,0090,"Óbuda, Bogdáni út",17
1747,048506,Szent István Bazilika,9140,"Káposztásmegyer, Mogyoródi-patak",30


## Sajnos nem világos miért nem lehet két mezőt könnyen selectezni logikai operátorokkal

In [44]:
#df_megallo_jaratok = pd.merge( df_stop_times[ df_stop_times["stop_id"] == stop_id and df_stop_times["arrival_time_min"] > 60*8+15 ] , df_trips, left_on="trip_id", right_on="trip_id").drop_duplicates("route_id")
#df_megallo_jaratok["stop_id"]=stop_id
#df_megallo_jaratok["stop_name"]=stop_name

#df_megallo_jaratok[["stop_id","stop_name","route_id","stop_headsign","stop_sequence"]]

In [45]:
 a = pd.Series(df_stop_times["stop_id"] == stop_id)
 a

0          False
1          False
2          False
3          False
4          False
           ...  
6181322    False
6181323    False
6181324    False
6181325    False
6181326    False
Name: stop_id, Length: 6181327, dtype: bool

In [46]:
 b = pd.Series(df_stop_times["arrival_time_min"] > 60*8+15 )
 b

0          False
1          False
2           True
3           True
4           True
           ...  
6181322    False
6181323    False
6181324    False
6181325    False
6181326    False
Name: arrival_time_min, Length: 6181327, dtype: bool

In [47]:
a & b

0          False
1          False
2          False
3          False
4          False
           ...  
6181322    False
6181323    False
6181324    False
6181325    False
6181326    False
Length: 6181327, dtype: bool

## Melyik járatok állnak meg ebben a megállóban reggel 9:00 körül (1)

In [48]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time.str.startswith("09").values'), df_trips, left_on='trip_id', right_on='trip_id').sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "arrival_time_min","stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,arrival_time_min,stop_headsign,stop_sequence
108,048506,Szent István Bazilika,0090,09:00:00,540,NaN,17
42,048506,Szent István Bazilika,4720,09:00:00,540,Zugló vasútállomás,9
36,048506,Szent István Bazilika,90,09:05:00,545,"Óbuda, Bogdáni út",17


## Melyik járatok állnak meg ebben a megállóban reggel 10:00 körül (2)


In [49]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time > "10:00:00"'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "arrival_time_min", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,arrival_time_min,stop_headsign,stop_sequence
3,048506,Szent István Bazilika,4720,10:01:00,601,Zugló vasútállomás,9
679,048506,Szent István Bazilika,0090,10:05:00,605,"Óbuda, Bogdáni út",17
491,048506,Szent István Bazilika,90,10:06:00,606,"Óbuda, Bogdáni út",17
1250,048506,Szent István Bazilika,F4720,10:32:00,632,Zugló vasútállomás,9
1248,048506,Szent István Bazilika,9500,24:09:00,1449,"Rákospalota, Székely Elek út",12
1246,048506,Szent István Bazilika,9140,24:19:00,1459,"Káposztásmegyer, Mogyoródi-patak",30


## Most inkább nem string-et keresünk, hanem számot
Kétszer olyan gyors lett a futtatás

In [50]:
df_megallo_jaratok = pd.merge( df_stop_times.query(f'stop_id == "{stop_id}" and arrival_time_min > 60*10'), df_trips, left_on="trip_id", right_on="trip_id").sort_values(["arrival_time"]).drop_duplicates("route_id")
df_megallo_jaratok["stop_id"]=stop_id
df_megallo_jaratok["stop_name"]=stop_name

df_megallo_jaratok[["stop_id","stop_name","route_id","arrival_time", "arrival_time_min", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,arrival_time_min,stop_headsign,stop_sequence
3,048506,Szent István Bazilika,4720,10:01:00,601,Zugló vasútállomás,9
679,048506,Szent István Bazilika,0090,10:05:00,605,"Óbuda, Bogdáni út",17
491,048506,Szent István Bazilika,90,10:06:00,606,"Óbuda, Bogdáni út",17
1250,048506,Szent István Bazilika,F4720,10:32:00,632,Zugló vasútállomás,9
1248,048506,Szent István Bazilika,9500,24:09:00,1449,"Rákospalota, Székely Elek út",12
1246,048506,Szent István Bazilika,9140,24:19:00,1459,"Káposztásmegyer, Mogyoródi-patak",30


## Most kiegészítjük az útvonal azonosítóval is (járatszám)

Ez még egy join

In [51]:
df_megallo_jaratok2 = pd.merge( df_megallo_jaratok, df_routes, left_on="route_id", right_on="route_id")
df_megallo_jaratok2[["stop_id","stop_name","route_id","arrival_time", "route_short_name", "stop_headsign","stop_sequence"]]

,stop_id,stop_name,route_id,arrival_time,route_short_name,stop_headsign,stop_sequence
0,048506,Szent István Bazilika,4720,10:01:00,72,Zugló vasútállomás,9
1,048506,Szent István Bazilika,0090,10:05:00,9,"Óbuda, Bogdáni út",17
2,048506,Szent István Bazilika,F4720,10:32:00,72❆,Zugló vasútállomás,9
3,048506,Szent István Bazilika,9500,24:09:00,950,"Rákospalota, Székely Elek út",12
4,048506,Szent István Bazilika,9140,24:19:00,914,"Káposztásmegyer, Mogyoródi-patak",30


In [52]:
stop_id = "F00933"
stop_id = "F01029"

parent_stop_id = df_stops[ df_stops["stop_id"] == stop_id]["parent_station"].iloc[0]
parent_stop_id

nan

Melyek azok az állomások amiknek van főmegállója?

# KÖZELI ÁTSZÁLLÁSI PONTOK

Ez úgy tűnik földrajzi koordináták alapján kell megkeresni.

In [53]:

# Create a list of Point objects
points = [Point(xy) for xy in zip(df_stops['stop_lon'], df_stops['stop_lat'])]

# Create a GeoSeries from the list of points with the correct CRS
gs = gpd.GeoSeries(points, crs='EPSG:4326')

gdf_stops = gpd.GeoDataFrame(df_stops, geometry=gs, crs="EPSG:4326")
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (19.1357 47.50037)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (19.12789 47.48314)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (19.10341 47.50037)
3,003105,Bécsi út / Vörösvári út,47.548817,19.029722,003105,NaN,NaN,NaN,2.0,POINT (19.02972 47.54882)
4,003117,Hubay Jenő tér,47.562900,19.115709,003117,NaN,NaN,NaN,1.0,POINT (19.11571 47.5629)


Az EPSG:4326 nem jó távolságmérésre, mert nem tudom miért.
De a 3035 az jó, Németországhoz kötött valami. Ezért átmegyek arra:

In [54]:
gdf_stops.to_crs("epsg:3035",inplace=True)
gdf_stops.head(5)

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
0,002133,"Örs vezér tere M+H, déli tárolótér",47.500366,19.135700,002133,NaN,NaN,NaN,NaN,POINT (5007781.301 2752100.781)
1,002138,Kőbánya alsó vasútállomás,47.483139,19.127891,002138,NaN,NaN,NaN,2.0,POINT (5007426.257 2750125.866)
2,003002,Puskás Ferenc Stadion M,47.500368,19.103406,003002,NaN,NaN,NaN,NaN,POINT (5005367.638 2751801.574)
3,003105,Bécsi út / Vörösvári út,47.548817,19.029722,003105,NaN,NaN,NaN,2.0,POINT (4999223.77 2756473.426)
4,003117,Hubay Jenő tér,47.562900,19.115709,003117,NaN,NaN,NaN,1.0,POINT (5005458.179 2758821.276)


Most a Nyugatihoz viszonyítva fogunk közeli megállókat keresni

In [55]:
stop_id = "F01029" # Nyugati
gdf_nyugati = gdf_stops [ gdf_stops["stop_id"] == stop_id ]
gdf_megallo = gdf_nyugati.to_crs(epsg=3035)
gdf_megallo

,stop_id,stop_name,stop_lat,stop_lon,stop_code,location_type,location_sub_type,parent_station,wheelchair_boarding,geometry
2537,F01029,Nyugati pályaudvar M,47.509806,19.057151,F01029,NaN,NaN,NaN,1.0,POINT (5001786.02 2752416.904)


In [56]:
gdf_nearest_stops = gdf_megallo.sjoin_nearest(gdf_stops,distance_col="distance", how="right", exclusive = True, max_distance = 500)
gdf_nearest_stops.columns
#gdf_nearest_stops[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]


Index(['index_left', 'stop_id_left', 'stop_name_left', 'stop_lat_left',
       'stop_lon_left', 'stop_code_left', 'location_type_left',
       'location_sub_type_left', 'parent_station_left',
       'wheelchair_boarding_left', 'stop_id_right', 'stop_name_right',
       'stop_lat_right', 'stop_lon_right', 'stop_code_right',
       'location_type_right', 'location_sub_type_right',
       'parent_station_right', 'wheelchair_boarding_right', 'geometry',
       'distance'],
      dtype='object')

In [57]:
gdf_nearest_stops.nsmallest(40,'distance')[["stop_id_left","stop_name_left","stop_lat_left","stop_lon_left","stop_id_right","stop_name_right","stop_lat_right","stop_lon_right","distance"]]

,stop_id_left,stop_name_left,stop_lat_left,stop_lon_left,stop_id_right,stop_name_right,stop_lat_right,stop_lon_right,distance
2461,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00935,Nyugati pályaudvar M,47.509922,19.056658,39.253332
6061,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYI,Nyugati pályaudvar [I],47.509870,19.056404,56.669957
2460,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00934,Nyugati pályaudvar M,47.509896,19.056394,57.835671
6057,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYE,Nyugati pályaudvar [E],47.510447,19.056426,89.626772
6060,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYH,Nyugati pályaudvar [H],47.510233,19.056103,91.944253
2459,F01029,Nyugati pályaudvar M,47.509806,19.057151,F00933,Nyugati pályaudvar M,47.510300,19.056167,92.071854
6059,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYG,Nyugati pályaudvar [G],47.510326,19.056196,92.107868
6052,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NY3,Nyugati pályaudvar (lift » M3),47.510040,19.055784,106.075445
127,F01029,Nyugati pályaudvar M,47.509806,19.057151,008137,Nyugati pályaudvar,47.510571,19.056072,117.418620
6058,F01029,Nyugati pályaudvar M,47.509806,19.057151,LM3NYF,Nyugati pályaudvar [F],47.510703,19.056257,120.137919
